In [3]:
from delphi import Delphi as DB
from creds import my_creds
db = DB()
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
import tensorflow as tf
import re
from gensim.models import Word2Vec
from gensim.models.phrases import Phraser, Phrases
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from gensim.test.utils import common_texts
from sklearn.utils import shuffle

In [ ]:
icd_10_data = pd.read_fwf('icd10cm_codes_2020.txt',header=None, names=["icd_10_code", "icd_10_title"])
print(icd_10_data)
icd_10_data['icd_10_code_first_digits']=icd_10_data['icd_10_code'].str[:3]
print(icd_10_data)
icd_10_data['icd_10_code_first_digits']=icd_10_data['icd_10_code'].str[:3]
merged_inner = icd_10_data
print(merged_inner)
doctor_notes_exp=merged_inner[['icd_10_title','icd_10_code_first_digits']]
print(doctor_notes_exp)


In [ ]:
def hier_text_preprocessing(df):
    texts=[]
    all_sentences = []
    df_grouped = df.groupby(['icd_10_code_first_digits']).agg({'icd_10_title': lambda x: ' '.join(x)})
    #print(df_grouped)
    for index in range(len(df_grouped)):
        texts.append(df_grouped.icd_10_title[index])
    print(texts[:5])
    sentences = []
    for index in range(len(texts)):
        sentences = [re.sub(pattern=r'[\!"#$%&\*+,-./:;<=>?@^_`()|~=]', 
                            repl='', 
                            string=x
                           ).strip().split(' ') for x in texts[index].split('\n') 
                     if not x.endswith('writes:')]
        """ sentences = [re.sub(pattern=r'[0-9]+', 
                            repl='', 
                            string=x
                           ).strip().split(' ') for x in texts[index].split('\n') 
                     if not x.endswith('writes:')]"""
        #sentences = ''.join([i for i in sentences if not i.isdigit()])
        sentences = [x for x in sentences if x != ['']]
        sentences=[[string.lower() for string in sublist] for sublist in sentences]
        sentences=[[re.sub('[^A-Za-z0-9]+', '', string) for string in sublist] for sublist in sentences]
        texts[index] = sentences
    for text in texts:
        all_sentences += text
    return all_sentences

In [ ]:
all_sentences=hier_text_preprocessing(doctor_notes_exp) 
#so all_sentences is a nested list consisting of words,
#in which case I'll need to decide which hierarchies to preserve, or i could try out the performance and see
#the problem is that, some of them don't have low level codes. maybe try use level 1 and level 2 code titles
#for experiments
print(all_sentences[:5])

In [4]:
common_terms =stopwords.words('english')
common_terms=common_terms+['unspecified','due','involvement','manifestation','west','nile','identified']
print(common_terms)
phrases = Phrases(all_sentences, common_terms=common_terms)
# The Phraser object is used from now on to transform sentences
bigram = Phraser(phrases)

# Applying the Phraser to transform our sentences is simply
all_sentences = list(bigram[all_sentences])
all_sentences
print(common_terms)
phrases = Phrases(all_sentences, common_terms=common_terms)
# The Phraser object is used from now on to transform sentences
bigram = Phraser(phrases)

# Applying the Phraser to transform our sentences is simply
all_sentences = list(bigram[all_sentences])
all_sentences

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

'phrases = Phrases(all_sentences, common_terms=common_terms)\n# The Phraser object is used from now on to transform sentences\nbigram = Phraser(phrases)\n\n# Applying the Phraser to transform our sentences is simply\nall_sentences = list(bigram[all_sentences])\nall_sentences\nprint(common_terms)\nphrases = Phrases(all_sentences, common_terms=common_terms)\n# The Phraser object is used from now on to transform sentences\nbigram = Phraser(phrases)\n\n# Applying the Phraser to transform our sentences is simply\nall_sentences = list(bigram[all_sentences])\nall_sentences'

In [ ]:
#test_data=[['cholera','cholera','vibrio','cholerae','biovar', 'elto', 'cholera','vibrio','cholerae','biovar', 'cholerae'],['infectious','gastroenteritis','colitis']]
#hier_icd_10_titles=hier_text_preprocessing(icd_10_data)
#common_terms =stopwords.words('english')
#common_terms=common_terms+['unspecified','due','involvement','manifestation','west','nile','identified']
#phrases = Phrases(hier_icd_10_titles, common_terms=common_terms)
# The Phraser object is used from now on to transform sentences
#bigram = Phraser(phrases)

# Applying the Phraser to transform our sentences is simply
#all_sentences = list(bigram[hier_icd_10_titles])
#print(all_sentences[:5])
model = Word2Vec(all_sentences, 
                 min_count=3,   # Ignore words that appear less than this
                 size=50,      # Dimensionality of word embeddings
                 workers=4,     # Number of processors (parallelisation)
                 sg=1,
                 hs=1,
                 negative=5,
                 ns_exponent=0.75,
                 cbow_mean=0.5,
                 window=5,      # Context window for words during training
                 iter=30,
                 compute_loss=True)       # Number of epochs training over corpus
model

In [ ]:
#print(model.wv.vectors.shape)
"""model.vector_size
len(model.wv.vocab)"""
#model.most_similar('pain unspecified')
#print(model.wv.most_similar('vibrio'))
#print(model['vibrio'])
#model.wv.most_similar('unspecified')
"""print(model.wv.most_similar('salmonella'))
print(model.wv.most_similar('rabies'))"""
model.get_latest_training_loss()
model.save("notes_word2vec_model")